In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import numpy as np
import pandas as pd
import re

**Explicación del código**

El código de Python que se muestra a continuación extrae información de juegos de mesa del sitio web "https://www.ludonauta.es" a través de Selenium y BeautifulSoup. 

Utiliza listas de páginas y funciones para obtener detalles sobre cada juego, incluyendo nombre, fecha de publicación, autores, categorías, mecánicas, puntuación, precios y disponibilidad en tiendas. 

Finalmente, se crean dos archivos JSON para guardar la información de juegos y precios, y se incorporan en dos DataFrames.

**Desafíos y resolución**

Se presentaron desafíos durante el web scraping al intentar extraer la información, se abordaron mediante soluciones específicas que detallo a continuación:
-	En el HTML muchas de las etiquetas estaban nombradas igual, Se sacaron las etiquetas anteriores o posteriores que las contenían, y luego utilizar los métodos FIND : NEXT y PREVIOUS SIBLING para encontrar los elementos buscados.

-	Al extraer los datos se hizo limpieza, eliminando tabulaciones o mediante expresiones regulares para encontrar patrones.

-	Cuando se extrajeron los precios, también se extraían los nombres de las tiendas y la disponibilidad de los juegos relacionados que aparecían abajo, por eso se mapearon las tiendas con el tamaño de los precios. 

-	Se guardaron los datos de los precios en listas  y no en diccionarios, para extraer todos los precios de venta.

### LISTADO DE LAS PÁGINAS 

A continuación, se crea un listado completo de las páginas concatenando el sitio base con números del 1 al 379 (número total de páginas que hay en la web en el momento de extracción).

In [ ]:
url_ludonauta= "https://www.ludonauta.es"

listado_paginas= []
for numero_pagina in range(1,380): 
    todas_paginas= f"{url_ludonauta}/juegos-mesas/listar/page:{numero_pagina}"
    numero_pagina += 1
    listado_paginas.append(todas_paginas)

### EXTRACCIÓN DEL LINK DE LOS JUEGOS DE CADA PAGINA

La función extraer_href_pagina recibe un objeto BeautifulSoup (soup) que representa la página web de cada juego. Busca todos los enlaces de juegos con la clase "product-name", construye las URL completas al agregarles el prefijo url_ludonauta, y devuelve la lista de URL de cada juego.

In [ ]:
def extraer_href_pagina(soup):
    """Crea una lista de los enlaces_juegos de cada página"""
    juegos_href = soup.find_all("a", class_="product-name")

    enlaces_juegos = []

    for juego in juegos_href:
        enlace = juego.get('href')
        enlace_entero= url_ludonauta + enlace

        enlaces_juegos.append(enlace_entero)
    
    return enlaces_juegos

### EXTRACCIÓN DE LA INFORMACIÓN DE CADA JUEGO 


La función extraer_juego(soup), recibe un objeto BeautifulSoup que representa la web de Ludonauta. Luego, utiliza métodos de búsqueda para extraer información específica sobre el juego. La información se organiza en un diccionario llamado json_juego, que luego se devuelve como resultado de la función. Si algún dato no está disponible, se asigna el valor np.nan.

In [ ]:
def extraer_juego(soup):
    """Extrae de los juegos: Nombre, Fecha publicación, Autores, Categorías, 
    Mecánicas, Puntuación, Jugadores, J.Mínimos y J.Máximos (si no los hay, son Jugadores), 
    Duración, D.Mínima y D.Máxima (si no la hay, es Duración), Edad, Complejidad y Dep idioma,
    y lo guarda en json_juego"""

    try:
        nombre_juego= soup.find("h2").text
    except:
        nombre_juego= np.nan
    try:
        puntuacion= soup.find("span", class_="text-navy").text
    except:
        puntuacion= np.nan
    try:
        contenedor_fecha = soup.find('dt', string='Fecha pub.')
        etiquetas_dentro_fecha = contenedor_fecha.find_next_sibling('dd').find_all('p')
        for etiqueta in etiquetas_dentro_fecha:
            fecha_publicacion= etiqueta.text
    except:
        fecha_publicacion= np.nan
    try:
        contenedor_autores = soup.find('dt', string='Autores')
        etiquetas_dentro_autores = contenedor_autores.find_next_sibling('dd').find_all('p')
        for etiqueta in etiquetas_dentro_autores:
            autores_sin_limpiar= etiqueta.text.split("|")
            autores = [autor.strip() for autor in autores_sin_limpiar]
    except:
        autores= np.nan
    try:
        contenedor_categorias = soup.find('dt', string='Categorías')
        etiquetas_dentro_categorias = contenedor_categorias.find_next_sibling('dd').find_all('p')
        for etiqueta in etiquetas_dentro_categorias:
            categorias_sin_limpiar= etiqueta.text.split("|")
            categorias = [categoria.strip() for categoria in categorias_sin_limpiar]
    except:
        categorias= np.nan
    try:
        contenedor_mecanicas = soup.find('dt', string='Mecánicas')
        etiquetas_dentro_mecanicas = contenedor_mecanicas.find_next_sibling('dd').find_all('p')
        for etiqueta in etiquetas_dentro_mecanicas:
            mecanicas_sin_limpiar= etiqueta.text.split("|")
            mecanicas = [mecanica.strip() for mecanica in mecanicas_sin_limpiar]
    except:
        mecanicas= np.nan
    try:
        contenedor_complejidad = soup.find('dt', string='Complejidad')
        etiquetas_dentro_complejidad = contenedor_complejidad.find_next_sibling('dd').find_all('span', class_='label label-navy')
        # COMPLEJIDAD (1: muy baja, 2: baja, 3: media, 4: alta, 5: muy alta)
        complejidad= len(etiquetas_dentro_complejidad)
    except:
        complejidad= np.nan
    try:
        contenedor_idioma = soup.find('dt', string='Dep. idioma')
        etiquetas_dentro_idioma = contenedor_idioma.find_next_sibling('dd').find_all('span', class_='label label-navy')
        # DEP IDIOMA (1: muy baja, 2: baja, 3: media, 4: alta, 5: muy alta)
        dep_idioma= len(etiquetas_dentro_idioma)
    except:
        dep_idioma= np.nan
    try:
        contenedor_jugadores = soup.find('small', string='jugadores')
        etiquetas_dentro_jugadores = contenedor_jugadores.find_previous_sibling('div').text
        jugadores_lista = [x.replace('\t', '').strip() for x in etiquetas_dentro_jugadores.split("\n") if x.strip() != ""]
        jugadores= str(jugadores_lista[0])
    except:
        jugadores= np.nan
    try:
        jugadores_minimos = [elemento.split('-')[0] for elemento in jugadores_lista]
        jugadores_minimos= (int(jugadores_minimos[0]))
    except:
        jugadores_minimos= np.nan
    try:
        jugadores_maximos = jugadores_lista[0].split('-')[1] if '-' in jugadores_lista[0] else jugadores_minimos
        jugadores_maximos = int(jugadores_maximos)
    except:
        jugadores_maximos= np.nan
    try:
        contenedor_duracion = soup.find('small', string='minutos')
        etiquetas_dentro_duracion = contenedor_duracion.find_previous_sibling('div').text
        juego = [x.replace('\t', '').strip() for x in etiquetas_dentro_duracion.split("\n") if x.strip() != ""]
        duracion_juego= str(juego[0])
        if duracion_juego == "-":
            duracion_juego= np.nan
    except:
        duracion_juego= np.nan
    try:
        duracion_minima = [elemento.split('-')[0] for elemento in juego]
        duracion_minima= int(duracion_minima[0])
    except:
        duracion_minima= np.nan
    try:
        duracion_maxima = juego[0].split('-')[1] if '-' in juego[0] else duracion_minima
        duracion_maxima = int(duracion_maxima)
    except:
        duracion_maxima= np.nan
    try:
        contenedor_edad = soup.find('small', string='años')
        etiquetas_dentro_edad = contenedor_edad.find_previous_sibling('div').text
        edad = [x.replace('\t', '').strip().replace("+", "") for x in etiquetas_dentro_edad.split("\n") if x.strip() != ""]
        edad= int(edad[0])
    except:
        edad= np.nan
    try:
        ediciones_juegos = soup.find("div", class_= "product-type small").text
        limpieza_elementos_juego = [x.replace('\t', '').strip() for x in ediciones_juegos.split("\n") if x.strip() != ""]
        str_ediciones_juegos= str(limpieza_elementos_juego[0])
    except:
        edicion_juegos= np.nan
        juegos_relacionados = np.nan
    try:
        edicion_juegos= str_ediciones_juegos.split(" «")[0]
    except:
        edicion_juegos= np.nan
    try:        
        juegos_relacionados= re.findall(r"«(.*?)»", str_ediciones_juegos)
        # no entraba en except porque cuando no había nada era una lista vacía
        if not juegos_relacionados:  
            juegos_relacionados = np.nan
    except:
        juegos_relacionados= np.nan
    json_juego= {"Nombre": nombre_juego, "Fecha publicación": fecha_publicacion, "Autores": autores, "Categorías": categorias, "Mecánicas": mecanicas, "Puntuación": puntuacion, "Jugadores": jugadores, "Jugadores mínimos": jugadores_minimos, "Jugadores máximos": jugadores_maximos, "Duración": duracion_juego, "Duración mínima": duracion_minima, "Duración máxima": duracion_maxima, "Edad": edad, "Complejidad": complejidad, "Dep idioma": dep_idioma, "Edición juegos": edicion_juegos, "Juegos relacionados": juegos_relacionados}

    return json_juego

### EXTRACCIÓN DE LOS PRECIOS, TIENDAS Y DISPONIBILIDAD DE LOS JUEGOS

La función precio_juegos(soup) toma el objeto soup que representa la página web de cada juego de Ludonauta, y después, busca y extrae información sobre los precios y la disponibilidad del juego en diferentes tiendas.

In [ ]:
def precio_juegos(soup):
    """Extrae de los juegos: lista de precios, lista de tiendas y su disponibilidad,
    y lo guarda en json_precios"""
    try:
        precios_principales= soup.find_all("span", class_= "product-price btn btn-sm btn-primary")
        lista_precios_1=[]
        for precio in precios_principales:
            precios_tiendas_principales= precio.text
            precios_principales= re.findall(r"\d+\,\d+", precios_tiendas_principales)
            lista_precios_1.extend(precios_principales)

        precios_secundarios= soup.find_all("span", class_= "product-price btn btn-sm btn-default")
        lista_precios_2=[]
        for precio in precios_secundarios:
            precios_tiendas_secundarios= precio.text
            precios_secundarios= re.findall(r"\d+\,\d+", precios_tiendas_secundarios)
            lista_precios_2.extend(precios_secundarios)
           
        lista_precios= lista_precios_1 + lista_precios_2
        if not lista_precios:
            lista_precios= np.nan
    except:
        lista_precios= np.nan
    try:
        tiendas= soup.find_all("a", class_="visible-xxs-inline-block visible-xs-inline-block visible-sm-inline-block visible-md-inline-block visible-lg-inline-block visible-xl-inline-block" )
        lista_tiendas=[]
        for tienda in tiendas:
            texto_tiendas = tienda.get('title')
            nombre_tienda = re.findall(r"«(.*?)»", texto_tiendas)
            lista_tiendas.extend(nombre_tienda)
            lista_tiendas = lista_tiendas[:len(lista_precios)]
            if not lista_tiendas:
                lista_tiendas= np.nan
    except:
        lista_tiendas= np.nan
    try:
        stock_contenedor= soup.find_all('td', class_='text-center')
        disponibilidad_juego= []
        for stock in stock_contenedor:
            en_stock= stock.text.replace('\n', '').strip()
            if en_stock: 
                disponibilidad_juego.append(en_stock)
                disponibilidad_juego= disponibilidad_juego[:len(lista_precios)]
                if not disponibilidad_juego:
                    disponibilidad_juego= np.nan
    except:
        disponibilidad_juego= np.nan
    
    json_precios={"Nombre tienda": lista_tiendas,"Precio": lista_precios, "Disponibilidad": disponibilidad_juego}

    return json_precios

### AUTOMATIZACIÓN DE LA EXTRACCIÓN DE DATOS 

Este script utiliza Selenium y BeautifulSoup para navegar a través de la lista de páginas web, extraer enlaces, acceder a cada enlace, y extraer información sobre juegos y precios. Los resultados se almacenan en diccionarios.

In [ ]:
browser = webdriver.Firefox()    
diccionario_juegos= {}
diccionario_precios= {}

# BUCLE PARA ACCEDER A CADA PÁGINA
for i in range(0,379): # Número de páginas de la web
    cada_pagina= browser.get(listado_paginas[i])                                                        
    soup = BeautifulSoup(browser.page_source, "html.parser")
    sleep(2)
    enlaces = extraer_href_pagina(soup)
    
    # EXTRACCIÓN DEL LINK DE LOS JUEGOS DE CADA PAGINA
    for enlace in enlaces:
        cada_enlace = browser.get(enlace)
        soup = BeautifulSoup(browser.page_source, "html.parser")
        
        # EXTRACCIÓN DE LA INFORMACIÓN DE CADA JUEGO 
        extraccion_juego= extraer_juego(soup)
        extraccion_precio= precio_juegos(soup)
            
        diccionario_juegos[enlace] = extraccion_juego
        diccionario_precios[enlace] = extraccion_precio

    sleep(2)

browser.quit()

### CSV DE JUEGOS

Se observa que ha recogido todos los datos y después, con pandas, se convierte diccionario_juegos en un DataFrame y se guarda en un archivo CSV.

In [ ]:
print(len(diccionario_juegos))

In [ ]:
df_juegos = pd.DataFrame.from_dict(diccionario_juegos, orient='index')
df_juegos.to_csv("juegos_ludonauta.csv")

### CSV DE PRECIOS

Se observa que ha recogido bien los precios, se convierte diccionario_precios en un DataFrame y se guarda en un CSV.

In [ ]:
print(len(diccionario_precios))

In [ ]:
df_precios = pd.DataFrame.from_dict(diccionario_precios, orient='index')
df_precios.to_csv("precios_ludonauta.csv")